In [119]:
from pandas.io import data, wb
import datetime
import os.path
import math
import pandas as pd
import numpy




In [120]:
df_train = pd.read_csv('data/spy-ohlc.csv', index_col='Date', parse_dates=True)
df_dividends = pd.read_csv('data/spy-dividends.csv', index_col='Date', parse_dates=True)
df_train = df_train.join(df_dividends)
df_train['dividend'].fillna(0, inplace=True)

In [121]:
pma10 = df_train['Close'].rolling(window=10,center=False).mean() + df_train['dividend'].rolling(window=10, center=False).mean()
pma20 = df_train['Close'].rolling(window=20,center=False).mean()+ df_train['dividend'].rolling(window=20, center=False).mean()
pma50 = df_train['Close'].rolling(window=50,center=False).mean()+ df_train['dividend'].rolling(window=50, center=False).mean()
pma200 = df_train['Close'].rolling(window=200,center=False).mean()+ df_train['dividend'].rolling(window=200, center=False).mean()
vma50 = df_train['Volume'].rolling(window=50, center=False).mean()



In [122]:
#distance from MAs
diffma10  = (df_train['Close']/pma10 -1)*100
diffma20 = (df_train['Close']/pma20 -1)*100
diffma50 = (df_train['Close']/pma50 -1)*100
diffma200 = (df_train['Close']/pma200 -1)*100

#tangential of ma lines
pma10tan = (pma10[1:]/pma10[:-1].values-1)*100
pma20tan = (pma20[1:]/pma20[:-1].values-1)*100
pma50tan = (pma50[1:]/pma50[:-1].values-1)*100
pma200tan = (pma200[1:]/pma200[:-1].values-1)*100

#sequence of higher high/higher low
hhx1 = df_train['High'][1:] + df_train['dividend'][1:] > df_train['High'][:-1] 
hlx1 = df_train['Low'][1:] + df_train['dividend'][1:] > df_train['Low'][:-1] 
hhx2 = hh.shift(1)
hhx3 = hh.shift(2)
hlx2 = hl.shift(1)
hlx3 = hl.shift(2)

#rolling average of hh and hl
hh10 = hh.rolling(window=10, center=False).mean()
hl10 = hl.rolling(window=10, center=False).mean()

nvx1 = df_train['Volume']/vma50


In [123]:
#add features for training data

#distance from MAs
df_train = df_train.assign(diffma10=diffma10)
df_train = df_train.assign(diffma20=diffma20)
df_train = df_train.assign(diffma50=diffma50)
df_train = df_train.assign(diffma200=diffma200)

#tangential of ma lines
df_train = df_train.assign(pma10tan=pma10tan)
df_train = df_train.assign(pma20tan=pma20tan)
df_train = df_train.assign(pma50tan=pma50tan)
df_train = df_train.assign(pma200tan=pma200tan)

df_train = df_train.assign(hhx1=hhx1)
df_train = df_train.assign(hlx1=hlx1)
df_train = df_train.assign(hhx2=hhx2)
df_train = df_train.assign(hhx3=hhx3)
df_train = df_train.assign(hlx2=hlx2)
df_train = df_train.assign(hlx3=hlx3)

df_train = df_train.assign(hh10=hh10)
df_train = df_train.assign(hl10=hl10)

#last day volume
df_train = df_train.assign(nvx1=nvx1)

In [124]:
remove = ['Open','High','Low','Close','Volume','Adj Close','dividend']
df_train.drop(remove, axis=1)
df_train.to_csv("data/spy-train.csv")

In [130]:
def predict(X_train, y_train, X_test):  
    from sklearn.cross_validation import train_test_split
    from sklearn import cross_validation
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import SVC


    #model = LogisticRegression(random_state=1)
    
    #model = RandomForestClassifier(n_estimators=350)
    #model = GradientBoostingClassifier(n_estimators=100)
    #best so far
    model = KNeighborsClassifier(n_neighbors=15)
    
    
    from sklearn import metrics
    
    #y_eval_comp = y_eval_pred[:,1] > 0.55
    #numpy.count_nonzero(y_eval_comp)
    
    #print("recall:", metrics.recall_score(y_eval, y_eval_comp, average='binary'))
    #print("precision:", metrics.precision_score(y_eval, y_eval_comp, average='binary'))
    #print("f1 score:", metrics.f1_score(y_eval, y_eval_comp, average='binary'))
    
    scores = cross_validation.cross_val_score(model, X_train, y_train, cv=5, scoring='precision')
    print("Precision score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))
    
    model.fit(X_train, y_train)
    y_test_pred = model.predict_proba(X_test);
    print(y_test_pred)
    return



In [142]:
X_train = df_train.values[201:-1, :]
X_test = X_train[-1:,:]
#print(len(X_train))
#for hh, shift one position back to get future value for hh
#print(hh.shift(-1).tail())
y_train = hh.shift(-1).values[201:].astype(bool)
#print(len(y_train))
print("predict hh")
e_predict(X_train, y_train, X_test)

#for hl
print("predict hl")
y_train = hl.shift(-1).values[201:].astype(bool)
e_predict(X_train, y_train, X_test)


predict hh


AttributeError: 'LogisticRegression' object has no attribute '__name__'

In [156]:
def e_predict(X_train, y_train, X_test):
    from sklearn import cross_validation
    from sklearn.linear_model import LogisticRegression
    from sklearn.naive_bayes import GaussianNB
    from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.ensemble import VotingClassifier

    clf1 = LogisticRegression(random_state=1)
    clf2 = RandomForestClassifier(n_estimators=350)
    clf3 = GaussianNB()
    clf4 = GradientBoostingClassifier(n_estimators=100)
    clf5 = KNeighborsClassifier(n_neighbors=15)
    
    eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3), 
                                        ('gb', clf4), ('knn',clf5)], voting='soft')

    for clf in list[clf1, clf2, clf3, clf4, clf5, eclf]:                         :
        print(type(clf).__name__)
        scores = cross_validation.cross_val_score(clf, X_train, y_train, cv=5, scoring='precision')
        print("Score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std())
        clf.fit(X_train, y_train)
        y_test_pred = clf.predict_proba(X_test);
        print(y_test_pred)


    return


SyntaxError: invalid syntax (<ipython-input-156-139913d05a52>, line 18)

In [152]:
from sklearn.linear_model import LogisticRegression
clf1 = LogisticRegression(random_state=1)
na = type(clf1).__name__
print(na)
    

LogisticRegression
